In [1]:
import pandas as pd

In [2]:
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

In [3]:
confirmed = pd.read_csv(url_confirmed) 
deaths = pd.read_csv(url_deaths) 
recovered = pd.read_csv(url_recovered)

In [4]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/27/22,1/28/22,1/29/22,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22,2/4/22,2/5/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,161004,161057,161290,162111,162926,163555,164190,164727,165358,165711
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,252577,254126,254126,255741,258543,258543,261240,261240,263172,263172
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,245698,247568,249310,250774,252117,253520,254885,255836,256806,257598
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,35028,35556,35556,35556,35958,35958,36315,36470,36599,36599
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,97901,98029,98057,98076,98116,98226,98267,98319,98340,98351


In [5]:
total_confirmed = confirmed.melt(
    id_vars=['Province/State','Country/Region','Lat','Long'],
    value_vars=confirmed.columns[4:],
    var_name='date',
    value_name='confirmed')

total_deaths = deaths.melt(
    id_vars=['Province/State','Country/Region','Lat','Long'],
    value_vars=deaths.columns[4:],
    var_name='date',
    value_name='deaths')

total_recovered = recovered.melt(
    id_vars=['Province/State','Country/Region','Lat','Long'],
    value_vars=recovered.columns[4:],
    var_name='date',
    value_name='recovered')
total_confirmed.tail()

,Province/State,Country/Region,Lat,Long,date,confirmed
208875,NaN,Vietnam,14.058324,108.277199,2/5/22,2327859
208876,NaN,West Bank and Gaza,31.952200,35.233200,2/5/22,556550
208877,NaN,Yemen,15.552727,48.516388,2/5/22,11149
208878,NaN,Zambia,-13.133897,27.849332,2/5/22,306777
208879,NaN,Zimbabwe,-19.015438,29.154857,2/5/22,230402


In [6]:
covid_data = total_confirmed.merge(right= total_deaths, how='left', on = ['Province/State','Country/Region','Lat','Long','date'])

In [7]:
covid_data = covid_data.merge(right= total_recovered, how='left', on = ['Province/State','Country/Region','Lat','Long','date'])

In [8]:
covid_data.head()

,Province/State,Country/Region,Lat,Long,date,confirmed,deaths,recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.15330,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0,0,0.0


In [9]:
covid_data.date = pd.to_datetime(covid_data.date)
covid_data.recovered = covid_data.recovered.fillna(0)

In [10]:
covid_data.isna().sum()

Province/State    143978
Country/Region         0
Lat                 1492
Long                1492
date                   0
confirmed              0
deaths                 0
recovered              0
dtype: int64

In [11]:
covid_data['active'] = covid_data.confirmed - covid_data.deaths - covid_data.recovered

In [12]:
covid_data.tail()

,Province/State,Country/Region,Lat,Long,date,confirmed,deaths,recovered,active
208875,NaN,Vietnam,14.058324,108.277199,2022-02-05,2327859,38261,0.0,2289598.0
208876,NaN,West Bank and Gaza,31.952200,35.233200,2022-02-05,556550,5128,0.0,551422.0
208877,NaN,Yemen,15.552727,48.516388,2022-02-05,11149,2026,0.0,9123.0
208878,NaN,Zambia,-13.133897,27.849332,2022-02-05,306777,3928,0.0,302849.0
208879,NaN,Zimbabwe,-19.015438,29.154857,2022-02-05,230402,5362,0.0,225040.0


In [13]:
covid_data.date.iloc[-1].strftime('%B %d , %Y')

'February 05 , 2022'

In [14]:
covid_data_1 = covid_data.groupby(['date'])[['confirmed','deaths','recovered','active']].sum().reset_index()
covid_data_1

,date,confirmed,deaths,recovered,active
0,2020-01-22,557,17,30.0,510.0
1,2020-01-23,655,18,32.0,605.0
2,2020-01-24,941,26,39.0,876.0
3,2020-01-25,1434,42,42.0,1350.0
4,2020-01-26,2118,56,56.0,2006.0
...,...,...,...,...,...
741,2022-02-01,381756438,5689337,0.0,376067101.0
742,2022-02-02,384944189,5701431,0.0,379242758.0
743,2022-02-03,388109962,5712940,0.0,382397022.0
744,2022-02-04,391129461,5726321,0.0,385403140.0


In [15]:
covid_data_1['confirmed'].iloc[-1]

393217243

In [16]:
round(
    ((covid_data_1['confirmed'].iloc[-1] - covid_data_1['confirmed'].iloc[-2])/covid_data_1['confirmed'].iloc[-1])*100,2)

0.53

In [17]:
covid_data_1.recovered.max()

129357529.0

In [18]:
covid_data_1['recovered'].nlargest(2).iloc[0]

129357529.0

In [19]:
covid_data['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Guatemala', 'Guin

In [20]:
covid_data_2 = covid_data.groupby(['date', 'Country/Region'])[['confirmed','deaths','recovered','active']].sum().reset_index()
covid_data_3 = covid_data_2[covid_data_2['Country/Region']== 'Brazil'][['Country/Region','date','confirmed']].reset_index()
covid_data_3['daily confirmed'] = covid_data_3['confirmed'] - covid_data_3['confirmed'].shift(1)

In [21]:
covid_data_3

,index,Country/Region,date,confirmed,daily confirmed
0,23,Brazil,2020-01-22,0,NaN
1,219,Brazil,2020-01-23,0,0.0
2,415,Brazil,2020-01-24,0,0.0
3,611,Brazil,2020-01-25,0,0.0
4,807,Brazil,2020-01-26,0,0.0
...,...,...,...,...,...
741,145259,Brazil,2022-02-01,25634781,171251.0
742,145455,Brazil,2022-02-02,25820745,185964.0
743,145651,Brazil,2022-02-03,26107894,287149.0
744,145847,Brazil,2022-02-04,26326454,218560.0


In [22]:
covid_data_3['Rolling ave.'] = covid_data_3['daily confirmed'].rolling(window=7).mean()

In [24]:
covid_data_4 = covid_data.groupby(['Lat','Long', 'Country/Region'])[['confirmed','deaths','recovered','active']].sum().reset_index()
covid_data_4

,Lat,Long,Country/Region,confirmed,deaths,recovered,active
0,-51.7963,-59.5236,United Kingdom,29628,0,14754.0,1.487400e+04
1,-42.8821,147.3272,Australia,854441,8653,101796.0,7.439920e+05
2,-40.9006,174.8860,New Zealand,2626632,18342,964499.0,1.643791e+06
3,-38.4161,-63.6167,Argentina,1850605862,39769717,711610324.0,1.099226e+09
4,-37.8136,144.9631,Australia,41311789,511003,7099128.0,3.370166e+07
...,...,...,...,...,...,...,...
272,64.2823,-135.0000,Canada,287230,2662,0.0,2.845680e+05
273,64.8255,-124.8457,Canada,374127,1521,0.0,3.726060e+05
274,64.9631,-19.0208,Iceland,6124543,16535,2058185.0,4.049823e+06
275,70.2998,-83.1076,Canada,258583,1407,0.0,2.571760e+05


In [25]:
covid_data_list = covid_data[['Country/Region','Lat','Long']]

In [27]:
covid_of_locations = covid_data_list.set_index(['Country/Region'])[['Lat','Long']].T.to_dict('dict')
covid_of_locations

/tmp/ipykernel_6428/2342774228.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  covid_of_locations = covid_data_list.set_index(['Country/Region'])[['Lat','Long']].T.to_dict('dict')


{'Afghanistan': {'Lat': 33.93911, 'Long': 67.709953},
 'Albania': {'Lat': 41.1533, 'Long': 20.1683},
 'Algeria': {'Lat': 28.0339, 'Long': 1.6596},
 'Andorra': {'Lat': 42.5063, 'Long': 1.5218},
 'Angola': {'Lat': -11.2027, 'Long': 17.8739},
 'Antigua and Barbuda': {'Lat': 17.0608, 'Long': -61.7964},
 'Argentina': {'Lat': -38.4161, 'Long': -63.6167},
 'Armenia': {'Lat': 40.0691, 'Long': 45.0382},
 'Australia': {'Lat': -31.9505, 'Long': 115.8605},
 'Austria': {'Lat': 47.5162, 'Long': 14.5501},
 'Azerbaijan': {'Lat': 40.1431, 'Long': 47.5769},
 'Bahamas': {'Lat': 25.025885, 'Long': -78.035889},
 'Bahrain': {'Lat': 26.0275, 'Long': 50.55},
 'Bangladesh': {'Lat': 23.685, 'Long': 90.3563},
 'Barbados': {'Lat': 13.1939, 'Long': -59.5432},
 'Belarus': {'Lat': 53.7098, 'Long': 27.9534},
 'Belgium': {'Lat': 50.8333, 'Long': 4.469936},
 'Belize': {'Lat': 17.1899, 'Long': -88.4976},
 'Benin': {'Lat': 9.3077, 'Long': 2.3158},
 'Bhutan': {'Lat': 27.5142, 'Long': 90.4336},
 'Bolivia': {'Lat': -16.2902